<div dir='rtl'>
<h1>پروژه اول</h1>
</div>

In [12]:
from timeit import default_timer as timer
from copy import deepcopy, copy
from collections import deque
import heapq

In [13]:
INPUT_COUNT = 6
INPUT_PATH  = 'data/input%d.txt'
TIMER_TEST_COUNT = 3

In [32]:
class Graph:
    def __init__(self, n, m):
        self.n = n
        self.m = m
        self.current = 0
        self.cost = 0
        self.parent = None
        self.adj = [set() for _ in range(n)]
        self.difficulties = {}
        self.requirements = {}
        self.remainingTime = 0

    def __deepcopy__(self, memo):
        g = Graph(self.n, self.m)
        g.current = self.current
        g.cost = self.cost
        g.parent = self.parent
        g.adj = copy(self.adj)
        g.difficulties = deepcopy(self.difficulties)
        g.requirements = deepcopy(self.requirements)
        g.remainingTime = self.remainingTime
        return g

    def __lt__(self, other):
        return self.cost < other.cost

    def __key(self):
        return (self.current, self.difficulties, self.requirements)

    def __str__(self):
        return str(self.__key())

In [15]:
def readInput(path):
    with open(path, 'r', encoding='utf-8') as f:
        n, m = map(int, f.readline().split())
        graph = Graph(n, m)
        for _ in range(m):
            u, v = map(int, f.readline().split())
            graph.adj[u - 1].add(v - 1)
            graph.adj[v - 1].add(u - 1)
        _ = int(f.readline())
        difficulties = map(int, f.readline().split())
        for d in difficulties:
            graph.difficulties[d - 1] = -1
        s = int(f.readline())
        for _ in range(s):
            inp = f.readline().split()
            p = int(inp[0]) - 1
            vertices = {int(x) - 1 for x in inp[2:]}
            graph.requirements[p] = vertices
        i = int(f.readline()) - 1
        graph.current = i
        if i in graph.difficulties:
            graph.difficulties[i] += 1
        for u, vertices in graph.requirements.items():
            if i in vertices:
                vertices.remove(i)
        if i in graph.requirements and len(graph.requirements[i]) == 0:
            del graph.requirements[i]
    return graph

In [33]:
inputs = [readInput(INPUT_PATH % i) for i in range(1, INPUT_COUNT + 1)]

In [17]:
def searchTime(g, method):
    sum = 0
    for _ in range(TIMER_TEST_COUNT):
        gCopy = deepcopy(g)
        start = timer()
        method(gCopy)
        end = timer()
        sum += end - start
    return sum / TIMER_TEST_COUNT

In [18]:
def moveInGraph(g, dest):
    if dest not in g.adj[g.current]:
        return None
    newGraph = deepcopy(g)
    newGraph.current = dest
    newGraph.cost += 1
    newGraph.parent = g
    if dest in newGraph.difficulties:
        newGraph.difficulties[dest] += 1
        newGraph.remainingTime = newGraph.difficulties[dest]
    for _, vertices in newGraph.requirements.items():
        if dest in vertices:
            vertices.remove(dest)
    if dest in newGraph.requirements and len(newGraph.requirements[dest]) == 0:
        newGraph.requirements.pop(dest)
    return newGraph


def isGoal(g):
    return len(g.requirements) == 0


def getPath(g):
    path = []
    while g is not None:
        path.append(g.current + 1)
        g = g.parent
    path.reverse()
    return path

In [19]:
def bfs(g):
    if isGoal(g):
        return getPath(g), g.cost,1
    discovered = set()
    queue = deque([g])
    discovered.add(str(g))
    while queue:
        g = queue.popleft()
        if g.remainingTime > 0:
            g.remainingTime -= 1
            g.cost += 1
            queue.append(g)
            continue
        for dest in g.adj[g.current]:
            newState = moveInGraph(g, dest)
            if newState is None:
                continue
            if isGoal(newState):
                return getPath(newState), newState.cost, len(discovered)
            if str(newState) not in discovered:
                queue.append(newState)
                discovered.add(str(newState))
    return None, None, len(discovered)

In [20]:
for i, g in enumerate(inputs):
    gCopy = deepcopy(g)
    print(f'BFS: Input {i + 1}')
    path, cost, stateCount = bfs(gCopy)
    if path is not None:
        print('Path:', ' -> '.join(map(str, path)))
        print('Cost:', cost)
        print('Total Visited States:', stateCount)
        print(f'Time: {searchTime(g, bfs):.4f} seconds')
    else:
        print('No solution')
    print()

BFS: Input 1
Path: 1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Cost: 8
Total Visited States: 78
Time: 0.0034 seconds

BFS: Input 2
Path: 1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Cost: 8
Total Visited States: 78
Time: 0.0034 seconds

BFS: Input 3
Path: 9 -> 10 -> 9 -> 4 -> 12 -> 3 -> 7 -> 5 -> 8
Cost: 8
Total Visited States: 115
Time: 0.0056 seconds

BFS: Input 4
Path: 13 -> 11 -> 10 -> 3 -> 2 -> 6 -> 12 -> 5 -> 9 -> 4 -> 1 -> 13 -> 11 -> 10
Cost: 13
Total Visited States: 1769
Time: 0.1332 seconds

BFS: Input 5
Path: 28 -> 19 -> 13 -> 3 -> 11 -> 24 -> 9 -> 2 -> 5 -> 7 -> 29 -> 22 -> 28
Cost: 12
Total Visited States: 7536
Time: 1.2215 seconds

BFS: Input 6
Path: 40 -> 42 -> 38 -> 24 -> 31 -> 45 -> 30 -> 48 -> 41 -> 18 -> 1 -> 19 -> 43 -> 49 -> 47 -> 49 -> 9 -> 34 -> 25 -> 50 -> 12 -> 16
Cost: 21
Total Visited States: 10630
Time: 1.0926 seconds



In [21]:
def dfs(g, depth, discovered, visited):
    if isGoal(g):
        return g
    if depth <= 0:
        return None
    remainingTime = g.remainingTime
    if remainingTime > 0:
        g.remainingTime = 0
        g.cost += remainingTime
        if depth <= remainingTime + 1:
            return None
    children = set()
    for dest in g.adj[g.current]:
        newState = moveInGraph(g, dest)
        if newState is None:
            continue
        if str(newState) in visited:
            continue
        children.add(((newState, str(newState))))
        discovered.add(str(newState))
        visited.add(str(newState))
    for child, _ in children:
        ans = dfs(child, depth - 1 - remainingTime, discovered, visited)
        if ans is not None:
            return ans
    
    for _, key in children:
        visited.remove(str(key))
    return None


def ids(g):
    depth = 0
    while True:
        discovered = set()
        discovered.add(str(g))
        visited = set()
        visited.add(str(g))
        ans = dfs(g, depth, discovered, visited)
        if ans is not None:
            return getPath(ans), ans.cost, len(discovered)
        # print(f'IDS: Depth {depth} - Visited States: {len(discovered)}')
        depth += 1

In [42]:
for i, g in enumerate(inputs[:4]): # remove [:4] to run all inputs (takes a long time =] )
    gCopy = deepcopy(g)
    print(f'IDS: Input {i + 1}')
    path, cost, stateCount = ids(gCopy)
    if path is not None:
        print('Path:', ' -> '.join(map(str, path)))
        print('Cost:', cost)
        print('Total Visited States:', stateCount)
        print(f'Time: {searchTime(g, ids):.4f} seconds')
    else:
        print('No solution')
    print()

IDS: Input 1
Path: 1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Cost: 8
Total Visited States: 22
Time: 0.0112 seconds

IDS: Input 2
Path: 1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Cost: 8
Total Visited States: 24
Time: 0.0230 seconds

IDS: Input 3
Path: 9 -> 10 -> 6 -> 4 -> 12 -> 3 -> 7 -> 5 -> 8
Cost: 8
Total Visited States: 89
Time: 0.0427 seconds

IDS: Input 4
Path: 13 -> 11 -> 10 -> 3 -> 2 -> 6 -> 12 -> 5 -> 9 -> 4 -> 1 -> 13 -> 11 -> 10
Cost: 13
Total Visited States: 1621
Time: 1.2681 seconds



In [30]:
def heuristic(g):
    remainingCount = set()
    for r, vertices in g.requirements.items():
        remainingCount.add(r)
        remainingCount.update(vertices)
    return len(remainingCount) + g.remainingTime

def aStar(g, alpha = 1):
    q = []
    heapq.heappush(q, (0, g))
    discovered = set()
    discovered.add(str(g))
    while q:
        _, g = heapq.heappop(q)
        if isGoal(g):
            return getPath(g), g.cost, len(discovered)
        if g.remainingTime > 0:
            g.remainingTime -= 1
            g.cost += 1
            heapq.heappush(q, (g.cost + alpha * heuristic(g), g))
            continue
        for dest in g.adj[g.current]:
            newState = moveInGraph(g, dest)
            if newState is None:
                continue
            if str(newState) not in discovered:
                heapq.heappush(q, (newState.cost + alpha * heuristic(newState), newState))
                discovered.add(str(newState))
    return None, None, len(discovered)

In [47]:
for i, g in enumerate(inputs):
    gCopy = deepcopy(g)
    print(f'A*: Input {i + 1}')
    path, cost, stateCount = aStar(gCopy)
    if path is not None:
        print('Path:', ' -> '.join(map(str, path)))
        print('Cost:', cost)
        print('Total Visited States:', stateCount)
        print(f'Time: {searchTime(g, lambda g: aStar(g, 1)):.4f} seconds')
    else:
        print('No solution')
    print()

A*: Input 1
Path: 1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Cost: 8
Total Visited States: 78
Time: 0.0022 seconds

A*: Input 2
Path: 1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Cost: 8
Total Visited States: 78
Time: 0.0070 seconds

A*: Input 3
Path: 9 -> 10 -> 2 -> 4 -> 12 -> 3 -> 7 -> 5 -> 8
Cost: 8
Total Visited States: 88
Time: 0.0042 seconds

A*: Input 4
Path: 13 -> 11 -> 10 -> 3 -> 2 -> 6 -> 12 -> 5 -> 9 -> 4 -> 1 -> 13 -> 11 -> 10
Cost: 13
Total Visited States: 711
Time: 0.0447 seconds

A*: Input 5
Path: 28 -> 19 -> 13 -> 3 -> 11 -> 24 -> 9 -> 27 -> 5 -> 7 -> 29 -> 22 -> 28
Cost: 12
Total Visited States: 2670
Time: 0.2911 seconds

A*: Input 6
Path: 40 -> 42 -> 38 -> 24 -> 31 -> 45 -> 30 -> 48 -> 41 -> 18 -> 1 -> 19 -> 43 -> 49 -> 47 -> 49 -> 9 -> 34 -> 25 -> 50 -> 12 -> 16
Cost: 21
Total Visited States: 7894
Time: 0.7808 seconds



In [49]:
for i, g in enumerate(inputs):
    gCopy = deepcopy(g)
    print(f'A* (alpha = 1.5): Input {i + 1}')
    path, cost, stateCount = aStar(gCopy, 1.5)
    if path is not None:
        print('Path:', ' -> '.join(map(str, path)))
        print('Cost:', cost)
        print('Total Visited States:', stateCount)
        print(f'Time: {searchTime(g, lambda g: aStar(g, 2)):.4f} seconds')
    else:
        print('No solution')
    print()

A* (alpha = 1.5): Input 1
Path: 1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Cost: 8
Total Visited States: 52
Time: 0.0012 seconds

A* (alpha = 1.5): Input 2
Path: 1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Cost: 8
Total Visited States: 52
Time: 0.0011 seconds

A* (alpha = 1.5): Input 3
Path: 9 -> 10 -> 2 -> 4 -> 12 -> 3 -> 7 -> 5 -> 8
Cost: 8
Total Visited States: 46
Time: 0.0015 seconds

A* (alpha = 1.5): Input 4
Path: 13 -> 11 -> 10 -> 3 -> 2 -> 6 -> 12 -> 5 -> 9 -> 4 -> 1 -> 13 -> 11 -> 10
Cost: 13
Total Visited States: 198
Time: 0.0038 seconds

A* (alpha = 1.5): Input 5
Path: 28 -> 19 -> 3 -> 11 -> 24 -> 9 -> 2 -> 5 -> 7 -> 29 -> 20 -> 13 -> 23 -> 28
Cost: 13
Total Visited States: 181
Time: 0.0041 seconds

A* (alpha = 1.5): Input 6
Path: 40 -> 42 -> 38 -> 24 -> 31 -> 45 -> 30 -> 48 -> 41 -> 18 -> 1 -> 19 -> 43 -> 49 -> 47 -> 49 -> 9 -> 34 -> 25 -> 50 -> 12 -> 16
Cost: 21
Total Visited States: 5563
Time: 0.3080 seconds



In [50]:
for i, g in enumerate(inputs):
    gCopy = deepcopy(g)
    print(f'A* (alpha = 2): Input {i + 1}')
    path, cost, stateCount = aStar(gCopy, 2)
    if path is not None:
        print('Path:', ' -> '.join(map(str, path)))
        print('Cost:', cost)
        print('Total Visited States:', stateCount)
        print(f'Time: {searchTime(g, lambda g: aStar(g, 2)):.4f} seconds')
    else:
        print('No solution')
    print()

A* (alpha = 2): Input 1
Path: 1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Cost: 8
Total Visited States: 44
Time: 0.0014 seconds

A* (alpha = 2): Input 2
Path: 1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Cost: 8
Total Visited States: 44
Time: 0.0012 seconds

A* (alpha = 2): Input 3
Path: 9 -> 10 -> 2 -> 4 -> 12 -> 3 -> 7 -> 5 -> 8
Cost: 8
Total Visited States: 38
Time: 0.0013 seconds

A* (alpha = 2): Input 4
Path: 13 -> 1 -> 4 -> 9 -> 2 -> 3 -> 10 -> 5 -> 12 -> 5 -> 10 -> 11 -> 10 -> 5 -> 9 -> 4
Cost: 15
Total Visited States: 86
Time: 0.0031 seconds

A* (alpha = 2): Input 5
Path: 28 -> 19 -> 3 -> 11 -> 24 -> 9 -> 2 -> 5 -> 7 -> 29 -> 20 -> 13 -> 23 -> 28
Cost: 13
Total Visited States: 112
Time: 0.0039 seconds

A* (alpha = 2): Input 6
Path: 40 -> 42 -> 38 -> 24 -> 31 -> 45 -> 30 -> 48 -> 41 -> 18 -> 1 -> 19 -> 43 -> 49 -> 47 -> 49 -> 9 -> 34 -> 25 -> 50 -> 12 -> 16
Cost: 21
Total Visited States: 3196
Time: 0.2550 seconds

